In [9]:
# Class Decision tree
class DecisionTree :
    class Node :
        def __init__(self) :
            self.feature = 0
            self.class_no = -1
            self.children = []
    
    def __init__(self) :
        import numpy
        self.np = numpy
    
    # Train on the dataset
    def train(self, file) :
        # Take dataset from file
        dataset = self.np.genfromtxt(file, delimiter=',', dtype="|S50", autostrip=True)
        self.np.random.shuffle(dataset)
        # Seperate class 0 and class 1 dataset and perform stratified splitting into train and validation
        dataset_0 = dataset[dataset[:,14] == '0']
        dataset_1 = dataset[dataset[:,14] == '1']
        trainset = self.np.concatenate((dataset_0[:4*len(dataset_0)//5], dataset_1[:4*len(dataset_1)//5]), axis=0)
        validationset = self.np.concatenate((dataset_0[4*len(dataset_0)//5:], dataset_1[4*len(dataset_1)//5:]), axis=0)
        # Shuffle train and validation
        self.np.random.shuffle(trainset)
        self.np.random.shuffle(validationset)
        # Preprocess train set
        train = self.sanitize(trainset)
        # Array to stroe number of children in each features that are possible
        self.no_children = self.np.empty(14, dtype=self.np.int)
        for i in xrange(14) :
            self.no_children[i] = self.np.amax(train[:,i])
        # Preprocess Valiodation set
        valid = self.sanitize_valid(validationset)
        valid[:,14] = validationset[:,14].astype(self.np.int)
        self.train = train
        self.valid_x = valid[:,:-1]
        self.valid_y = valid[:,-1]
        # Array to keep note of features that have been decied
        decided_features = self.np.zeros(14)
        # Learn recursively the tree
        node = self.learn_node(self.train, decided_features)
        self.node = node
        
    # Validate and predict the accuracy
    def validate(self) :
        correct_answers = 0
        for i in xrange(len(self.valid_x)) :
            if self.predict_row(self.valid_x[i]) == self.valid_y[i] :
                correct_answers += 1
        print "Accuracy is", float(correct_answers)/float(len(self.valid_x))
    
    # Predict on the given test_file and return the outputs
    def predict(self, test_file) :
        dataset = self.np.genfromtxt(test_file, delimiter=',', dtype="|S50", autostrip=True, invalid_raise=False)
        test_x = self.sanitize_valid(dataset)
        test_y = self.np.empty(len(test_x), dtype=self.np.int)
        for i in xrange(len(test_x)) :
            test_y[i] = self.predict_row(test_x[i,:])
        return test_y

    # Predict one test case
    def predict_row(self, test_row) :
        # Recursively go through the nodes until you find the answer
        node = self.node
        while node.class_no == -1 :
            fv = test_row[node.feature]
            if fv < len(node.children) :
                node = node.children[fv]
            else :
                node = node.children[-1]
        return node.class_no        
    
    # Determine the entropy on the given feature from given dataset
    def entropy(self, dataset, feature) :
        n = float(len(dataset))
        ent = 0.0
        for i in xrange(self.no_children[feature]+1) :
            ni = float(len(dataset[dataset[:,feature] == 1]))
            pi = ni/n
            if ni != 0.0 :
                ent += -pi*self.np.log2(pi)
        return ent
    
    # Learn nodes of decision tree recursively
    def learn_node(self, dataset, decided_features) :
        node = DecisionTree.Node()
        # If no dataset left predict randomly
        if len(dataset) == 0 :
            node.class_no = self.np.random.randint(0, 1)
            return node
        # If all datasets belong to class 1 predict 1
        if (len(dataset) == len(dataset[dataset[:,14] == 1])) :
            node.class_no = 1
            return node
        # If all datasets belong to class 1 predict 2
        elif (len(dataset) == len(dataset[dataset[:,14] == 0])) :
            node.class_no = 0
            return node
        # Calculate feature whose entropy is least
        minentropy = self.np.inf;
        minentropy_feature = -1
        for i in  xrange(14) :
            if decided_features[i] == 0 :
                ent = self.entropy(dataset, i)
                if ent < minentropy :
                    minentropy = ent
                    minentropy_feature = i
        # NO feature is foud then predict randomly and end
        if minentropy_feature == -1 :
            node.class_no = self.np.random.randint(0,1)
            return node
        # Split on feature with least entropy
        node.feature = minentropy_feature
        for i in xrange(self.no_children[i]+1) :
            new_decided_features = self.np.empty(14)
            new_decided_features[:] = decided_features
            new_decided_features[node.feature] = 1
            node.children.append(self.learn_node(dataset[dataset[:,node.feature] == i], new_decided_features))
        return node
     
    # Sanitize train data
    def sanitize(self, dataset) :
        self.replacer = []
        for i in xrange(len(dataset[0])) :
            u, indices = self.np.unique(dataset[:, i], return_inverse=True)
            self.replacer.append(u[self.np.argmax(self.np.bincount(indices))])
        for i in xrange(len(dataset[0])) :
            dataset[:,i][dataset[:,i] == '?'] = self.replacer[i]
        data = self.np.empty((len(dataset), len(dataset[0])), dtype=self.np.int)
        data[:,0] = dataset[:,0].astype(self.np.int)
        for i in xrange(len(data)) :
            data[i,0] = data[i,0] // 10
        self.col1dict, data[:,1] = self.np.unique(dataset[:, 1], return_inverse=True)
        data[:,2] = dataset[:,2].astype(self.np.int)
        for i in xrange(len(data)) :
            data[i,2] = data[i,2] // 25000
        self.col3dict, data[:,3] = self.np.unique(dataset[:, 3], return_inverse=True)
        data[:,4] = dataset[:,4].astype(self.np.int)
        self.col5dict, data[:,5] = self.np.unique(dataset[:, 5], return_inverse=True)
        self.col6dict, data[:,6] = self.np.unique(dataset[:, 6], return_inverse=True)
        self.col7dict, data[:,7] = self.np.unique(dataset[:, 7], return_inverse=True)
        self.col8dict, data[:,8] = self.np.unique(dataset[:, 8], return_inverse=True)
        self.col9dict, data[:,9] = self.np.unique(dataset[:, 9], return_inverse=True)
        data[:,10] = dataset[:,10].astype(self.np.int)
        for i in xrange(len(data)) :
            data[i,10] = data[i,10] // 5000
        data[:,11] = dataset[:,11].astype(self.np.int)
        for i in xrange(len(data)) :
            data[i,11] = data[i,11] // 100
        data[:,12] = dataset[:,12].astype(self.np.int)
        for i in xrange(len(data)) :
            data[i,12] = data[i,12] // 10
        self.col13dict, data[:,13] = self.np.unique(dataset[:, 13], return_inverse=True)
        data[:,14] = dataset[:,14].astype(self.np.int)
        return data
    
    # Sanitize test and validation data
    def sanitize_valid(self, dataset) :
        data = self.np.empty((len(dataset), len(dataset[0])), dtype=self.np.int)
        for i in xrange(len(dataset[0])) :
            dataset[dataset[:, i] == '?', i] = self.replacer[i]
        data[:,0] = dataset[:,0].astype(self.np.int)
        for i in xrange(len(data)) :
            data[i,0] = data[i,0] // 10
        for i in xrange(len(self.col1dict)) :
            data[:,1][dataset[:,1] == self.col1dict[i]] = i
        data[:,2] = dataset[:,2].astype(self.np.int)
        for i in xrange(len(data)) :
            data[i,2] = data[i,2] // 25000
        for i in xrange(len(self.col3dict)) :
            data[:,3][dataset[:,3] == self.col3dict[i]] = i
        data[:,4] = dataset[:,4].astype(self.np.int)
        for i in xrange(len(self.col5dict)) :
            data[:,5][dataset[:,5] == self.col5dict[i]] = i
        for i in xrange(len(self.col6dict)) :
            data[:,6][dataset[:,6] == self.col6dict[i]] = i
        for i in xrange(len(self.col7dict)) :
            data[:,7][dataset[:,7] == self.col7dict[i]] = i
        for i in xrange(len(self.col8dict)) :
            data[:,8][dataset[:,8] == self.col8dict[i]] = i
        for i in xrange(len(self.col9dict)) :
            data[:,9][dataset[:,9] == self.col9dict[i]] = i
        data[:,10] = dataset[:,10].astype(self.np.int)
        for i in xrange(len(data)) :
            data[i,10] = data[i,10] // 5000
        data[:,11] = dataset[:,11].astype(self.np.int)
        for i in xrange(len(data)) :
            data[i,11] = data[i,11] // 100
        data[:,12] = dataset[:,12].astype(self.np.int)
        for i in xrange(len(data)) :
            data[i,12] = data[i,12] // 10
        for i in xrange(len(self.col13dict)) :
            data[:,13][dataset[:,13] == self.col13dict[i]] = i
        return data

In [20]:
# Intialize decision tree, validate, pickle and save
dt = DecisionTree()
dt.train('data/train.csv')
dt.validate()

import dill
with open('cs15btech11031.model', 'wb') as file:
    dill.dump(dt, file)

Accuracy is 0.792603698151


In [21]:
with open('cs15btech11031.model', 'rb') as file:
    st = dill.load(file)
    st.validate()

Accuracy is 0.792603698151
